In [3]:
import csv
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from pathlib import Path

PRODUCT_URL = "https://www.amazon.com/Apple-Version-Orange-Unlocked-Renewed/dp/B0FTC2PRVZ/ref=mp_s_a_1_3?crid=12DU2W23NGUG6&dib=eyJ2IjoiMSJ9.GyiR0GcyN_tVX_yq3UHqDHSsfvKHHipt_aXtlfqAbuLAIApqsYWEOJrlXC8YgKT2U2oflmiHUMRRBM13CnJCq1jPBl8eHdakCgv4UFERp6RQlZk0x9cm4bp0MLfPcTHLNK1RAtVT67uM_W05Ttdfm5DL5YXmpHlBLsAhQCJ5MgWQRC9posVq_wNTSSqiMQBEOyrZbiTAp7lV-FsHRd1Xgw.mYf6M63S7Hoq-FcR-xtNBYOJdv5j-1s71GnP9vAZn7o&dib_tag=se&keywords=iphone%2B17%2Bpro%2Bmax&qid=1764770796&sprefix=Iphone%2B1%2Caps%2C190&sr=8-3&th=1"


def get_page_html(url: str) -> str:
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/123.0 Safari/537.36"
        ),
        "Accept-Language": "en-US,en;q=0.9",
    }

    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()
    return response.text


def parse_price_and_title(html: str) -> tuple[str | None, str | None]:
    soup = BeautifulSoup(html, "html.parser")

    title_tag = soup.find(id="productTitle")
    if title_tag:
        title = title_tag.get_text(strip=True)
    else:
        title = None

    price = None
    price_ids = [
        "priceblock_ourprice",
        "priceblock_dealprice",
        "priceblock_saleprice",
        "corePriceDisplay_desktop_feature_div",
    ]

    for pid in price_ids:
        tag = soup.find(id=pid)
        if tag and tag.get_text(strip=True):
            price = tag.get_text(strip=True)
            break

    if price is None:
        offscreen = soup.find("span", class_="a-offscreen")
        if offscreen:
            price = offscreen.get_text(strip=True)

    return title, price


def append_to_csv(timestamp: str, title: str | None, price: str | None, url: str,
                  filename: str = "price_history.csv") -> None:
    """
    Append one row of data to a CSV file. Creates the file with headers if it does not exist.
    """
    path = Path(filename)
    file_exists = path.is_file()

    with path.open("a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)

        if not file_exists:
            writer.writerow(["timestamp", "title", "price", "url"])

        writer.writerow([timestamp, title or "", price or "", url])


def track_product(url: str) -> None:
    """
    Fetch, display, and log the current price and title for one product URL.
    """
    try:
        html = get_page_html(url)
        title, price = parse_price_and_title(html)
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        print("===================================")
        print(f"Checked at: {timestamp}")
        print(f"URL: {url}")

        if title:
            print(f"Title: {title}")
        else:
            print("Title: could not be found")

        if price:
            print(f"Price: {price}")
        else:
            print("Price: could not be found")
        print("===================================")

        append_to_csv(timestamp, title, price, url)

    except requests.HTTPError as e:
        print(f"HTTP error: {e}")
    except requests.RequestException as e:
        print(f"Network error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")


def main():
    # No input needed, it uses your PRODUCT_URL
    track_product(PRODUCT_URL)


if __name__ == "__main__":
    main()


Checked at: 2025-12-04 14:37:38
URL: https://www.amazon.com/Apple-Version-Orange-Unlocked-Renewed/dp/B0FTC2PRVZ/ref=mp_s_a_1_3?crid=12DU2W23NGUG6&dib=eyJ2IjoiMSJ9.GyiR0GcyN_tVX_yq3UHqDHSsfvKHHipt_aXtlfqAbuLAIApqsYWEOJrlXC8YgKT2U2oflmiHUMRRBM13CnJCq1jPBl8eHdakCgv4UFERp6RQlZk0x9cm4bp0MLfPcTHLNK1RAtVT67uM_W05Ttdfm5DL5YXmpHlBLsAhQCJ5MgWQRC9posVq_wNTSSqiMQBEOyrZbiTAp7lV-FsHRd1Xgw.mYf6M63S7Hoq-FcR-xtNBYOJdv5j-1s71GnP9vAZn7o&dib_tag=se&keywords=iphone%2B17%2Bpro%2Bmax&qid=1764770796&sprefix=Iphone%2B1%2Caps%2C190&sr=8-3&th=1
Title: Apple iPhone 17 Pro Max, US Version, 256GB, eSIM, Cosmic Orange- Unlocked (Renewed)
Price: $1,369.97 with 9 percent savings-9%$1,369.97Typical price: $1,500.00Typical price:$1,500.00$1,500.00The Typical Price is determined using the 90-day median price paid by customers for the product in the Amazon store. We exclude prices paid by customers for the product when it has been on promotion for a limited time and prices paid in the Amazon Haul or Amazon Now store.Learn